In [1]:
# KAGGLE ---------------------
import sys
import numpy as np
import keras
from keras.utils import Sequence
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import os
import imgaug as ia
from imgaug import augmenters as iaa
import cv2

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import metrics
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import keras
import tensorflow as tf


/Users/farrar/py3.6.5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# MINE -----------------------------------
import hprotein

In [3]:
# KAGGLE ---------------------------------
BATCH_SIZE = 2
SEED = 777
SHAPE = (512, 512, 4)
DIR = '../input'
VAL_RATIO = 0.3 # 10 % as validation
THRESHOLD = 0.5 # due to different cost of True Positive vs False Positive, this is the probability threshold to predict the class as 'yes'

ia.seed(SEED)

In [4]:
# MINE -----------------------------------
# constants
TRAIN_PATH = '../stage1_train'
TEST_PATH = '../stage1_test'
LABEL_PATH = '../stage1_labels/train_short.csv'
OUTPUT_PATH = '../stage1_submit'
COLORS = ['red','green', 'blue', 'yellow']
IMAGE_SIZE = 512
CROP_SIZE = 512
BATCH_SIZE = 2
SHAPE = (CROP_SIZE, CROP_SIZE, 4)
THRESHOLD = 0.05
SEED = 42


In [5]:
# KAGGLE ---------------------
def getTrainDataset():
    
    # path_to_train = DIR + '/train/'
    # data = pd.read_csv(DIR + '/train.csv')

    path_to_train = TRAIN_PATH
    data = pd.read_csv(LABEL_PATH)
    
    paths = []
    labels = []
    
    for name, lbl in zip(data['Id'], data['Target'].str.split(' ')):
        y = np.zeros(28)
        for key in lbl:
            y[int(key)] = 1
        paths.append(os.path.join(path_to_train, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

# unit test ------------------------------------
p, l = getTrainDataset()
print (len(p))
print (p)
print (len(l))
print(type(l[0][1]))
print((l[0][1]))
print (l)

10
['../stage1_train/000c99ba-bba4-11e8-b2b9-ac1f6b6435d0'
 '../stage1_train/001838f8-bbca-11e8-b2bc-ac1f6b6435d0'
 '../stage1_train/001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/0020af02-bbba-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0'
 '../stage1_train/fb4c1fac-bbaa-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/fc84a97c-bbad-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/fea6e496-bbbb-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/ffeae6f0-bbc9-11e8-b2bc-ac1f6b6435d0'
 '../stage1_train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0']
10
<class 'numpy.float64'>
1.0
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.

In [6]:
# MINE -----------------------------------
p_me, l_me = hprotein.get_data(TRAIN_PATH, LABEL_PATH, mode='train', filter_ids=hprotein.mini_validate_set)
print (len(p_me))
print (p_me)
print (len(l_me))
print(type(l_me[0][1]))
print((l_me[0][1]))
print (l_me)

7
['000c99ba-bba4-11e8-b2b9-ac1f6b6435d0'
 '001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0'
 '0020af02-bbba-11e8-b2ba-ac1f6b6435d0'
 'fb4c1fac-bbaa-11e8-b2ba-ac1f6b6435d0'
 'fc84a97c-bbad-11e8-b2ba-ac1f6b6435d0'
 'fea6e496-bbbb-11e8-b2ba-ac1f6b6435d0'
 'fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0']
7
<class 'numpy.float64'>
1.0
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


In [32]:
# KAGGLE ---------------------
def getTestDataset():
    
    #path_to_test = DIR + '/test/'
    #data = pd.read_csv(DIR + '/sample_submission.csv')
    path_to_test = TEST_PATH
    data = pd.read_csv(OUTPUT_PATH + '/sample_submission_short.csv')


    paths = []
    labels = []
    
    for name in data['Id']:
        y = np.ones(28)
        paths.append(os.path.join(path_to_test, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

# unit test ------------------------------------
p_test, l_test = getTestDataset()
print (len(p_test))
print (p_test)
print (len(l_test))
print (l_test)

7
['../stage1_test/00cfafb0-bacb-11e8-b2b8-ac1f6b6435d0'
 '../stage1_test/00d2a4f8-bad6-11e8-b2b9-ac1f6b6435d0'
 '../stage1_test/000cce7e-bad4-11e8-b2b8-ac1f6b6435d0'
 '../stage1_test/0006faa6-bac7-11e8-b2b7-ac1f6b6435d0'
 '../stage1_test/0008baca-bad7-11e8-b2b9-ac1f6b6435d0'
 '../stage1_test/003170fa-bacd-11e8-b2b8-ac1f6b6435d0'
 '../stage1_test/0031820a-baca-11e8-b2b8-ac1f6b6435d0']
7
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  

In [8]:
# MINE -----------------------------------
p_me_test, l_me_test = hprotein.get_data(TEST_PATH, OUTPUT_PATH, mode='test', filter_ids=None)
print (len(p_me_test))
print (p_me_test)
print (len(l_me_test))
print (l_me_test)

7
['0008baca-bad7-11e8-b2b9-ac1f6b6435d0'
 '00cfafb0-bacb-11e8-b2b8-ac1f6b6435d0'
 '000cce7e-bad4-11e8-b2b8-ac1f6b6435d0'
 '0006faa6-bac7-11e8-b2b7-ac1f6b6435d0'
 '0031820a-baca-11e8-b2b8-ac1f6b6435d0'
 '00d2a4f8-bad6-11e8-b2b9-ac1f6b6435d0'
 '003170fa-bacd-11e8-b2b8-ac1f6b6435d0']
7
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]]


In [9]:
# KAGGLE -----------------------------------
class ProteinDataGenerator(keras.utils.Sequence):
            
    def __init__(self, paths, labels, batch_size, shape, shuffle = False, use_cache = False, augment = False):
        self.paths, self.labels = paths, labels
        self.batch_size = batch_size
        self.shape = shape
        self.shuffle = shuffle
        self.use_cache = use_cache
        self.augment = augment
        if use_cache == True:
            self.cache = np.zeros((paths.shape[0], shape[0], shape[1], shape[2]), dtype=np.float16)
            self.is_cached = np.zeros((paths.shape[0]))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]

        paths = self.paths[indexes]
        X = np.zeros((paths.shape[0], self.shape[0], self.shape[1], self.shape[2]))
        # Generate data
        if self.use_cache == True:
            X = self.cache[indexes]
            for i, path in enumerate(paths[np.where(self.is_cached[indexes] == 0)]):
                image = self.__load_image(path)
                self.is_cached[indexes[i]] = 1
                self.cache[indexes[i]] = image
                X[i] = image
        else:
            for i, path in enumerate(paths):
                X[i] = self.__load_image(path)

        y = self.labels[indexes]
        
        if self.augment == True:
            seq = iaa.Sequential([
                iaa.OneOf([
                    iaa.Fliplr(0.5), # horizontal flips
                    iaa.Crop(percent=(0, 0.1)), # random crops
                    # Small gaussian blur with random sigma between 0 and 0.5.
                    # But we only blur about 50% of all images.
                    iaa.Sometimes(0.5,
                        iaa.GaussianBlur(sigma=(0, 0.5))
                    ),
                    # Strengthen or weaken the contrast in each image.
                    iaa.ContrastNormalization((0.75, 1.5)),
                    # Add gaussian noise.
                    # For 50% of all images, we sample the noise once per pixel.
                    # For the other 50% of all images, we sample the noise per pixel AND
                    # channel. This can change the color (not only brightness) of the
                    # pixels.
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                    # Make some images brighter and some darker.
                    # In 20% of all cases, we sample the multiplier once per channel,
                    # which can end up changing the color of the images.
                    iaa.Multiply((0.8, 1.2), per_channel=0.2),
                    # Apply affine transformations to each image.
                    # Scale/zoom them, translate/move them, rotate them and shear them.
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-180, 180),
                        shear=(-8, 8)
                    )
                ])], random_order=True)
        
            X = np.concatenate((X, seq.augment_images(X), seq.augment_images(X), seq.augment_images(X)), 0)
            y = np.concatenate((y, y, y, y), 0)
        
        return X, y
    
    def on_epoch_end(self):
        
        # Updates indexes after each epoch
        self.indexes = np.arange(len(self.paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        for item in (self[i] for i in range(len(self))):
            yield item
            
    def __load_image(self, path):
        R = Image.open(path + '_red.png')
        G = Image.open(path + '_green.png')
        B = Image.open(path + '_blue.png')
        Y = Image.open(path + '_yellow.png')

        im = np.stack((
            np.array(R), 
            np.array(G), 
            np.array(B),
            np.array(Y)), -1)

        #print('Before resize -> {}'.format(im.shape))
        im = cv2.resize(im, (SHAPE[0], SHAPE[1]))
        #print('After resize -> {}'.format(im.shape))
        im = np.divide(im, 255)
        return im

In [10]:
# KAGGLE -----------------------------------
paths, labels = getTrainDataset()

# divide to 
keys = np.arange(paths.shape[0], dtype=np.int)  
np.random.seed(SEED)
np.random.shuffle(keys)
lastTrainIndex = int((1-VAL_RATIO) * paths.shape[0])

pathsTrain = paths[0:lastTrainIndex]
labelsTrain = labels[0:lastTrainIndex]
pathsVal = paths[lastTrainIndex:]
labelsVal = labels[lastTrainIndex:]

print(paths.shape, labels.shape)
print(pathsTrain.shape, labelsTrain.shape, pathsVal.shape, labelsVal.shape)
print(pathsTrain)
print(labelsTrain)
print(pathsVal)
print(labelsVal)

(10,) (10, 28)
(7,) (7, 28) (3,) (3, 28)
['../stage1_train/000c99ba-bba4-11e8-b2b9-ac1f6b6435d0'
 '../stage1_train/001838f8-bbca-11e8-b2bc-ac1f6b6435d0'
 '../stage1_train/001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/0020af02-bbba-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0'
 '../stage1_train/fb4c1fac-bbaa-11e8-b2ba-ac1f6b6435d0'
 '../stage1_train/fc84a97c-bbad-11e8-b2ba-ac1f6b6435d0']
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 

In [11]:
# MINE -----------------------------------
p_v_me, l_v_me = hprotein.get_data(TRAIN_PATH, LABEL_PATH, mode='mini-validate', filter_ids=hprotein.mini_validate_set)
print(p_me.shape, l_me.shape, p_v_me.shape, l_v_me.shape)
print(p_me)
print(l_me)
print(p_v_me)
print(l_v_me)

(7,) (7, 28) (3,) (3, 28)
['000c99ba-bba4-11e8-b2b9-ac1f6b6435d0'
 '001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0'
 '0020af02-bbba-11e8-b2ba-ac1f6b6435d0'
 'fb4c1fac-bbaa-11e8-b2ba-ac1f6b6435d0'
 'fc84a97c-bbad-11e8-b2ba-ac1f6b6435d0'
 'fea6e496-bbbb-11e8-b2ba-ac1f6b6435d0'
 'fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0']
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
['001838f8-bbca-11e8-b2bc-ac1f6b6435d0'
 '002daad6-bbc9-11e8-b2bc-ac1

In [12]:
# KAGGLE -----------------------------------
tg = ProteinDataGenerator(pathsTrain, labelsTrain, BATCH_SIZE, SHAPE, use_cache=False, augment=False, shuffle=False)
vg = ProteinDataGenerator(pathsVal, labelsVal, BATCH_SIZE, SHAPE, use_cache=False, shuffle=False)
print (tg[1][0].shape)
print (tg[1][1].shape)
print (tg[1])

(2, 512, 512, 4)
(2, 28)
(array([[[[0.02352941, 0.14117647, 0.        , 0.70196078],
         [0.04313725, 0.08627451, 0.03529412, 0.63137255],
         [0.03137255, 0.05490196, 0.        , 0.72941176],
         ...,
         [0.02745098, 0.1372549 , 0.78431373, 0.1254902 ],
         [0.01960784, 0.2       , 0.69019608, 0.15294118],
         [0.0627451 , 0.39607843, 0.46666667, 0.14509804]],

        [[0.05882353, 0.07843137, 0.00784314, 0.58823529],
         [0.0745098 , 0.10980392, 0.01176471, 0.30980392],
         [0.1372549 , 0.06666667, 0.        , 0.43529412],
         ...,
         [0.03137255, 0.08627451, 0.71764706, 0.10980392],
         [0.04313725, 0.2745098 , 0.52941176, 0.15294118],
         [0.        , 0.15686275, 0.78431373, 0.14117647]],

        [[0.09803922, 0.04313725, 0.        , 0.37254902],
         [0.09411765, 0.38039216, 0.00784314, 0.4745098 ],
         [0.06666667, 0.07058824, 0.00392157, 0.80784314],
         ...,
         [0.        , 0.09803922, 0.8901960

In [13]:
# MINE -----------------------------------
class MyArgs:
    train_folder = TRAIN_PATH
    batch_size = BATCH_SIZE
    validation_steps = 3
    
args = MyArgs()
training_generator = hprotein.HproteinDataGenerator(args, args.train_folder, p_me, l_me, augment=True)
validate_generator = hprotein.HproteinDataGenerator(args, args.train_folder, p_v_me, l_v_me, augment=False)
print (training_generator[0][0].shape)
print (training_generator[0][1].shape)
print (training_generator[0])

(2, 512, 512, 4)
(2, 28)
(array([[[[0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         ...,
         [0.04313725, 0.1254902 , 0.01568627, 0.07058824],
         [0.04705882, 0.2       , 0.16862745, 0.05490196],
         [0.        , 0.05882353, 0.37254902, 0.00784314]],

        [[0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         ...,
         [0.02352941, 0.14901961, 0.07058824, 0.06666667],
         [0.16470588, 0.31764706, 0.25098039, 0.02352941],
         [0.05098039, 0.18823529, 0.31764706, 0.00784314]],

        [[0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         ...,
         [0.10980392, 0.14901961, 0.0627451

In [14]:
# KAGGLE -----------------------------------
def f1(y_true, y_pred):
    #y_pred = K.round(y_pred)
    #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# unit test
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

ftest = f1(y_true, y_pred)
#ftest = tf.constant(5)

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(ftest))

0.9473683


In [15]:
# MINE -----------------------------------
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

my_ftest = hprotein.f1(y_true, y_pred)
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(my_ftest))

0.9473683


In [16]:
# KAGGLE -----------------------------------
def f1_loss(y_true, y_pred):
    
    #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1-K.mean(f1)

# unit test
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

ftest = f1_loss(y_true, y_pred)
#ftest = tf.constant(5)

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(ftest))

0.052631676


In [17]:
# MINE -----------------------------------
y_true = K.variable(np.ones(10, np.uint8))
y_pred = np.ones(10, np.uint8)
y_pred[0] = 0
y_pred = K.variable(y_pred)

my_ftest = hprotein.f1_loss(y_true, y_pred)
#ftest = tf.constant(5)

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print (sess.run(my_ftest))

0.052631676


In [18]:
# KAGGLE -----------------------------------
def create_model(input_shape):
    
    dropRate = 0.25
    
    init = Input(input_shape)
    x = BatchNormalization(axis=-1)(init)
    x = Conv2D(32, (3, 3))(x) #, strides=(2,2))(x)
    x = ReLU()(x)

    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ginp1 = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(ginp1)
    x = Conv2D(64, (3, 3), strides=(2,2))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ginp2 = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(ginp2)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    ginp3 = Dropout(dropRate)(x)
    
    gap1 = GlobalAveragePooling2D()(ginp1)
    gap2 = GlobalAveragePooling2D()(ginp2)
    gap3 = GlobalAveragePooling2D()(ginp3)
    
    x = Concatenate()([gap1, gap2, gap3])
    
    x = BatchNormalization(axis=-1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    x = Dense(28)(x)
    x = Activation('sigmoid')(x)
    
    model = Model(init, x)
    
    return model

# unit
model = create_model(SHAPE)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-03),
    metrics=['acc',hprotein.f1])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 4)  0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512, 512, 4)  16          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 510, 510, 32) 1184        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 510, 510, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [19]:
# MINE -----------------------------------
my_model = hprotein.create_model(model_name='gap_net_bn_relu')
my_model.compile(loss='binary_crossentropy', optimizer=Adam(1e-03), metrics=['acc', hprotein.f1])
my_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 4)  0                                            
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 512, 512, 4)  16          input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 510, 510, 32) 1184        batch_normalization_12[0][0]     
__________________________________________________________________________________________________
re_lu_8 (ReLU)                  (None, 510, 510, 32) 0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [20]:
# KAGGLE -----------------------------------
checkpoint = ModelCheckpoint('./base.model', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)


In [21]:
# MINE -----------------------------------
my_checkpoint = ModelCheckpoint('{}.model'.format('model_12345'),
                                 monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min', period=1)


In [22]:
# KAGGLE -----------------------------------
epochs = 5

use_multiprocessing = False # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 
workers = 1 # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 

hist = model.fit_generator(
    tg,
    steps_per_epoch=len(tg),
    validation_data=vg,
    validation_steps=8,
    epochs=epochs,
    use_multiprocessing=use_multiprocessing,
    workers=workers,
    verbose=1,
    callbacks=[checkpoint])

Epoch 1/5
4/4 [==============================] - 23s 6s/step - loss: 0.7091 - acc: 0.6714 - f1: 0.0488 - val_loss: 2.5108 - val_acc: 0.7262 - val_f1: 0.0439

Epoch 00001: val_loss improved from inf to 2.51078, saving model to ./base.model
Epoch 2/5
4/4 [==============================] - 17s 4s/step - loss: 0.6130 - acc: 0.7449 - f1: 0.0478 - val_loss: 1.2949 - val_acc: 0.7262 - val_f1: 0.0404

Epoch 00002: val_loss improved from 2.51078 to 1.29493, saving model to ./base.model
Epoch 3/5
4/4 [==============================] - 18s 4s/step - loss: 0.4909 - acc: 0.8459 - f1: 0.0575 - val_loss: 0.7066 - val_acc: 0.7500 - val_f1: 0.0238

Epoch 00003: val_loss improved from 1.29493 to 0.70663, saving model to ./base.model
Epoch 4/5
4/4 [==============================] - 17s 4s/step - loss: 0.4214 - acc: 0.8964 - f1: 0.0538 - val_loss: 0.4023 - val_acc: 0.8690 - val_f1: 0.0132

Epoch 00004: val_loss improved from 0.70663 to 0.40227, saving model to ./base.model
Epoch 5/5
4/4 [=================

In [23]:
# MINE -----------------------------------
my_hist = my_model.fit_generator(training_generator,
                               steps_per_epoch=len(training_generator),
                               validation_data=validate_generator,
                               validation_steps=8,
                               epochs=5,
                               use_multiprocessing=False,
#                               max_queue_size=128,
                               workers=1,
                               verbose=1,
                               callbacks=[my_checkpoint])

Epoch 1/5
4/4 [==============================] - 20s 5s/step - loss: 0.6921 - acc: 0.6531 - f1: 0.0639 - val_loss: 3.7729 - val_acc: 0.5952 - val_f1: 0.0126

Epoch 00001: val_loss improved from inf to 3.77287, saving model to model_12345.model
Epoch 2/5
4/4 [==============================] - 16s 4s/step - loss: 0.6091 - acc: 0.7495 - f1: 0.0557 - val_loss: 1.6211 - val_acc: 0.7143 - val_f1: 0.0231

Epoch 00002: val_loss improved from 3.77287 to 1.62110, saving model to model_12345.model
Epoch 3/5
4/4 [==============================] - 16s 4s/step - loss: 0.5387 - acc: 0.8184 - f1: 0.0573 - val_loss: 1.1879 - val_acc: 0.7500 - val_f1: 0.0112

Epoch 00003: val_loss improved from 1.62110 to 1.18793, saving model to model_12345.model
Epoch 4/5
4/4 [==============================] - 16s 4s/step - loss: 0.4430 - acc: 0.9138 - f1: 0.0563 - val_loss: 0.9279 - val_acc: 0.8452 - val_f1: 0.0139

Epoch 00004: val_loss improved from 1.18793 to 0.92794, saving model to model_12345.model
Epoch 5/5
4/

In [24]:
# MY DATA GENERATORS WITH KAGGLE MODEL -----------------------------------
checkpoint = ModelCheckpoint('./my_base.model', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)

epochs = 5

use_multiprocessing = False # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 
workers = 1 # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 

hist = model.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator),
    validation_data=validate_generator,
    validation_steps=8,
    epochs=epochs,
    use_multiprocessing=use_multiprocessing,
    workers=workers,
    verbose=1,
    callbacks=[checkpoint])

Epoch 1/5
4/4 [==============================] - 17s 4s/step - loss: 0.3630 - acc: 0.9378 - f1: 0.0387 - val_loss: 0.4302 - val_acc: 0.9286 - val_f1: 0.0036

Epoch 00001: val_loss improved from inf to 0.43018, saving model to ./my_base.model
Epoch 2/5
4/4 [==============================] - 22s 6s/step - loss: 0.3143 - acc: 0.9332 - f1: 0.0482 - val_loss: 0.4425 - val_acc: 0.9405 - val_f1: 0.0022

Epoch 00002: val_loss did not improve from 0.43018
Epoch 3/5
4/4 [==============================] - 19s 5s/step - loss: 0.2726 - acc: 0.9378 - f1: 0.0459 - val_loss: 0.4440 - val_acc: 0.9286 - val_f1: 0.0047

Epoch 00003: val_loss did not improve from 0.43018
Epoch 4/5
4/4 [==============================] - 18s 4s/step - loss: 0.2448 - acc: 0.9515 - f1: 0.0515 - val_loss: 0.4939 - val_acc: 0.9286 - val_f1: 0.0033

Epoch 00004: val_loss did not improve from 0.43018
Epoch 5/5
4/4 [==============================] - 22s 5s/step - loss: 0.2467 - acc: 0.9469 - f1: 0.0445 - val_loss: 0.5004 - val_acc

In [25]:
# KAGGLE -----------------------------------
from sklearn.metrics import f1_score as off1

def getOptimalT(mdl, fullValGen):
    lastFullValPred = np.empty((0, 28))
    lastFullValLabels = np.empty((0, 28))
    for i in tqdm(range(len(fullValGen))):
        im, lbl = fullValGen[i]
        scores = mdl.predict(im)
        lastFullValPred = np.append(lastFullValPred, scores, axis=0)
        lastFullValLabels = np.append(lastFullValLabels, lbl, axis=0)
    print(lastFullValPred.shape, lastFullValLabels.shape)

    rng = np.arange(0, 1, 0.001)
    f1s = np.zeros((rng.shape[0], 28))
    for j, t in enumerate(tqdm(rng)):
        for i in range(28):
            p = np.array(lastFullValPred[:, i] > t, dtype=np.int8)
            # scoref1 = K.eval(f1_score(fullValLabels[:,i], p, average='binary'))
            scoref1 = off1(lastFullValLabels[:, i], p, average='binary')
            f1s[j, i] = scoref1

    print(np.max(f1s, axis=0))
    print(np.mean(np.max(f1s, axis=0)))

    T = np.empty(28)
    for i in range(28):
        T[i] = rng[np.where(f1s[:, i] == np.max(f1s[:, i]))[0][0]]
    # print('Choosing threshold: ', T, ', validation F1-score: ', max(f1s))
    print(T)

    return T, np.mean(np.max(f1s, axis=0))

# unit test 
thresholds, f1_macro = getOptimalT(model, vg)
print('thresholds -> {}'.format(thresholds))
print('f1 macro   -> {}'.format(f1_macro))

  0%|          | 0/1000 [00:00<?, ?it/s]/Users/farrar/py3.6.5/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
  1%|          | 6/1000 [00:00<00:16, 59.10it/s]

(3, 28) (3, 28)


  2%|▏         | 15/1000 [00:00<00:13, 70.61it/s]/Users/farrar/py3.6.5/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|██████████| 1000/1000 [00:09<00:00, 106.44it/s]

[0.8 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  1.  0.  0. ]
0.09999999999999999
[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.001 0.    0.   ]
thresholds -> [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.001 0.    0.   ]
f1 macro   -> 0.09999999999999999


In [26]:
# MINE ----------------------------------------
my_thresholds, my_f1_macro = hprotein.get_max_fscore_matrix(model, vg)
print('thresholds -> {}'.format(my_thresholds))
print('f1 macro   -> {}'.format(my_f1_macro))

  0%|          | 0/1000 [00:00<?, ?it/s]/Users/farrar/py3.6.5/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
  0%|          | 5/1000 [00:00<00:20, 47.54it/s]

(3, 28) (3, 28)


  1%|▏         | 13/1000 [00:00<00:15, 62.41it/s]/Users/farrar/py3.6.5/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|██████████| 1000/1000 [00:09<00:00, 102.18it/s]

[0.8 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  1.  0.  0. ]
0.09999999999999999
[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.001 0.    0.   ]
thresholds -> [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.001 0.    0.   ]
f1 macro   -> 0.09999999999999999


In [82]:
# Kaggle Predict
bestModel = load_model('../models/model_ca200f_fine_tune.model', custom_objects={'f1': f1, 'f1_loss': f1_loss})
p_test, l_test = getTestDataset()
T = np.load('../models/model_ca200f_fine_tune_thresh.npy')

print (len(p_test))
print (p_test)
print (len(l_test))
#print (l_test)

pathsTest = p_test
labelsTest = l_test

testg = ProteinDataGenerator(pathsTest, labelsTest, BATCH_SIZE, SHAPE)
submit = pd.read_csv('../stage1_submit/sample_submission_short.csv')
P = np.zeros((pathsTest.shape[0], 28))
for i in tqdm(range(len(testg))):
    images, labels = testg[i]
    score = bestModel.predict(images)
    P[i*BATCH_SIZE:i*BATCH_SIZE+score.shape[0]] = score
    


  0%|          | 0/4 [00:00<?, ?it/s]

7
['../stage1_test/00cfafb0-bacb-11e8-b2b8-ac1f6b6435d0'
 '../stage1_test/00d2a4f8-bad6-11e8-b2b9-ac1f6b6435d0'
 '../stage1_test/000cce7e-bad4-11e8-b2b8-ac1f6b6435d0'
 '../stage1_test/0006faa6-bac7-11e8-b2b7-ac1f6b6435d0'
 '../stage1_test/0008baca-bad7-11e8-b2b9-ac1f6b6435d0'
 '../stage1_test/003170fa-bacd-11e8-b2b8-ac1f6b6435d0'
 '../stage1_test/0031820a-baca-11e8-b2b8-ac1f6b6435d0']
7


100%|██████████| 4/4 [00:06<00:00,  1.70s/it]


In [38]:
print(P)

[[9.82032180e-01 1.40867790e-03 1.59820469e-04 3.66725399e-05
  2.44520546e-04 1.47103448e-04 2.30542361e-03 6.95718278e-04
  1.42989132e-09 1.07559572e-10 5.93328013e-14 1.27453264e-03
  2.43581948e-03 9.18872713e-04 6.16045290e-05 2.26510029e-08
  6.74355924e-02 4.51296847e-03 6.87073451e-04 3.49327992e-03
  3.46455025e-04 4.57480289e-02 3.75627391e-02 9.83743276e-03
  1.44700723e-04 5.21763265e-01 5.57976365e-02 5.11006419e-07]
 [2.72339303e-02 7.36814982e-05 2.09475635e-03 1.62891322e-03
  1.75289810e-03 9.63745639e-04 2.97165900e-01 9.88106243e-04
  7.56508598e-07 1.28239259e-08 4.73354212e-12 5.78880534e-02
  9.59956460e-03 1.19251767e-02 1.22812926e-04 1.33004996e-09
  1.10593741e-03 3.89207526e-05 3.17532074e-04 1.17047317e-03
  1.13920483e-04 1.03853576e-01 4.70100455e-02 4.08556223e-01
  2.07659741e-06 3.12625378e-01 4.78453608e-03 2.51691432e-07]
 [1.50032789e-01 3.78631894e-03 2.32310733e-03 5.73038764e-04
  4.61017853e-03 2.41895374e-02 3.79693210e-02 4.13314670e-01
  3.70

In [43]:
PP = np.array(P)
prediction = []

for row in tqdm(range(submit.shape[0])):

    str_label = ''

    for col in range(PP.shape[1]):
        if (PP[row, col] < T[col]):
            str_label += ''
        else:
            str_label += str(col) + ' '
    prediction.append(str_label.strip())

submit['Predicted'] = np.array(prediction)
submit.head(7)

100%|██████████| 7/7 [00:00<00:00, 5158.14it/s]


,Id,Predicted
0,00cfafb0-bacb-11e8-b2b8-ac1f6b6435d0,0 25 27
1,00d2a4f8-bad6-11e8-b2b9-ac1f6b6435d0,23 25 27
2,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,7 27
3,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25 27
4,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 27
5,003170fa-bacd-11e8-b2b8-ac1f6b6435d0,2 25 27
6,0031820a-baca-11e8-b2b8-ac1f6b6435d0,27


In [58]:
import logging

def get_best_model(args):

    final_model = None
    max_thresholds_matrix = None

    # Get thresholds
    logging.info('Getting correct model and thresholds...')
    if os.path.isfile('../models/{}_thresh.npy'.format(args.model_label)):

        # load model
        logging.info('Loading model {}...'.format(args.model_label))
        final_model = load_model('../models/{}.model'.format(args.model_label), custom_objects={'f1': f1})

        # load thresholds
        max_thresholds_matrix = np.load('../models/{}_thresh.npy'.format(args.model_label))

    elif os.path.isfile('../models/{}_fine_tune_thresh.npy'.format(args.model_label)):

        # load model
        logging.info('Loading model {}_fine_tune...'.format(args.model_label))
        final_model = load_model('../models/{}_fine_tune.model'.format(args.model_label),
                                 custom_objects={'f1': f1, 'f1_loss': f1_loss})

        # load thresholds
        max_thresholds_matrix = np.load('../models/{}_fine_tune_thresh.npy'.format(args.model_label))
    else:
        logging.warning("Can't find model file models/{}.model or models/{}_fine_tune.model".format(args.model_label,
                                                                                                    args.model_label))

    if max_thresholds_matrix is not None:
        logging.info('Using the following thresholds:')
        logging.info(max_thresholds_matrix)

    return final_model, max_thresholds_matrix


In [77]:
class MyArgs:
    train_folder = TRAIN_PATH
    batch_size = BATCH_SIZE
    validation_steps = 3
    submission_folder = '../stage1_submit'
    predict_folder = '../stage1_test'
    model_label = 'model_ca200f'
    model_name = 'gap_net_bn_relu'
    
args = MyArgs()

In [80]:
# Log start of predict process
print('Starting run_predict...')

# get the best model and related threshold matrix
final_model, max_thresholds_matrix = get_best_model(args)

# get predict data
print('Reading predict test set from {}...'.format(args.predict_folder))
predict_set_sids, predict_set_lbls = hprotein.get_data(args.predict_folder, args.submission_folder, mode='test',
                                                       filter_ids=[])
print(predict_set_sids)
predict_generator = hprotein.HproteinDataGenerator(args,
                                                   args.predict_folder,
                                                   predict_set_sids,
                                                   predict_set_lbls,
                                                   model_name=args.model_name)

# generate predictions
print('Starting prediction run...')

# read in the list of samples in the correct order to submit
submit = pd.read_csv('{}/sample_submission_short.csv'.format(args.submission_folder))

# create an empty array to catch the predictions
predictions = np.zeros((predict_set_sids.shape[0], 28))

# get the predictions
for i in range(len(predict_generator)):
    if i % 10 == 0:
        print('Predicting batch {} of {}'.format(i, len(predict_generator)))
    images, labels = predict_generator[i]
    score = final_model.predict(images)
    predictions[i * predict_generator.batch_size : ((i * predict_generator.batch_size) + score.shape[0])] = score
    


Starting run_predict...
Reading predict test set from ../stage1_test...
['0008baca-bad7-11e8-b2b9-ac1f6b6435d0'
 '00cfafb0-bacb-11e8-b2b8-ac1f6b6435d0'
 '000cce7e-bad4-11e8-b2b8-ac1f6b6435d0'
 '0006faa6-bac7-11e8-b2b7-ac1f6b6435d0'
 '0031820a-baca-11e8-b2b8-ac1f6b6435d0'
 '00d2a4f8-bad6-11e8-b2b9-ac1f6b6435d0'
 '003170fa-bacd-11e8-b2b8-ac1f6b6435d0']
Starting prediction run...
Predicting batch 0 of 4


In [79]:
# convert the predictions into the submission file format
prediction_str = []
for row in range(submit.shape[0]):
    if row % 200 == 0:
        print('Converting labels for prediction {} of {}'.format(row, submit.shape[0]))
    str_label = ''
    for col in range(predictions.shape[1]):
        if predictions[row, col] < max_thresholds_matrix[col]:
            str_label += ''
        else:
            str_label += str(col) + ' '
    prediction_str.append(str_label.strip())

# add column to pandas dataframe for submission
submit['Predicted'] = np.array(prediction_str)
submit.head(7)


Converting labels for prediction 0 of 7


,Id,Predicted
0,00cfafb0-bacb-11e8-b2b8-ac1f6b6435d0,0 27
1,00d2a4f8-bad6-11e8-b2b9-ac1f6b6435d0,0 25 27
2,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,7 27
3,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25 27
4,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,27
5,003170fa-bacd-11e8-b2b8-ac1f6b6435d0,23 25 27
6,0031820a-baca-11e8-b2b8-ac1f6b6435d0,2 25 27
